In [1]:
import json
from convertbng.util import convert_bng, convert_lonlat

In [2]:
DATASET_TYPE = "Bus"

In [3]:
def get_vehicle_type(init_vehicle_type):
    if init_vehicle_type == "FERRY":
        return "ferry"
    else:
        return "bus"
    

In [4]:
def filter_buggy_route(data,route_number):
    data["journeys"] = {k: v for k, v in data["journeys"].items() if v["route_number"] != route_number}
    return data

In [5]:
with open('../../data/initial_datasets/bristol' + DATASET_TYPE +'FullInitialDataset.json', 'r') as file:
    data = filter_buggy_route(json.loads(file.read()), "635")

In [6]:
def create_stops_dataset(data):
    stops = data["locations"]
    journeys = data["journeys"]
    parsed_stops = {}
    for stop in stops:
        stop_gps_coords = convert_lonlat([int(stop["easting"])],[int(stop["northing"])])
        parsed_stops[stop["location"]] = {
                "id": stop["location"].strip(),
                "label": stop["name"],
                "lon": stop_gps_coords[0][0],
                "lat": stop_gps_coords[1][0],
                "stationType": [],
                "routes": [],
                "connections": []
        }
    #classic version for adding connections in stopsData
    for key, journey in journeys.items():
        for stop_id in range(0, len(journey["stops"])):
            connections = []
            current_stop = parsed_stops[journey["stops"][stop_id]["location"].strip()]

            current_stop["routes"] =\
            list({*current_stop["routes"], journey["route_number"].strip()})
            
            current_stop["stationType"] = list({*current_stop["stationType"], get_vehicle_type(journey["vehicle_type"])})
            

            if stop_id > 0:
                connections.append(journey["stops"][stop_id-1]["location"])
            if stop_id < len(journey["stops"])-1:
                connections.append(journey["stops"][stop_id+1]["location"])
            current_stop["connections"] = [connection.strip() for connection\
                                           in list(filter(lambda x: x != current_stop["id"], \
                                                      {*current_stop["connections"], *connections}))]
    return list(parsed_stops.values())

In [7]:
#classic version for creation of routesData
def create_routes_dataset(data):
    journeys = data["journeys"]
    parsed_routes = []
    for key, journey in journeys.items():
        parsed_routes.append({
                "id": journey["route_number"].strip(),
                "direction": journey["route_direction"],
                "routeName": journey["route_number"].strip(),
                "stops": [stop["location"].strip() for stop in journey["stops"]],
                "bankHolidays": journey["bank_holidays"],
                "routeType": get_vehicle_type(journey["vehicle_type"])
            })
    return parsed_routes

In [8]:
with open('../../data/bristol_parsed/initial/bristol'+DATASET_TYPE+'Stops.json', 'w') as outfile:  
    json.dump(create_stops_dataset(data), outfile)
with open('../../data/bristol_parsed/initial/bristol'+DATASET_TYPE+'Routes.json', 'w') as outfile:  
    json.dump(create_routes_dataset(data), outfile)